In [1]:
import volpick.data
from obspy.clients.fdsn import Client
from obspy import UTCDateTime
import pandas as pd
import time
import datetime
import logging
import volpick
import numpy as np

In [2]:
log_level = logging.INFO
logger = logging.getLogger("Convert")
logger.propagate = False
logger.setLevel(log_level)

stream_handler = logging.StreamHandler()
stream_handler.setFormatter(
    logging.Formatter("%(asctime)s | %(name)s | %(levelname)s | %(message)s")
)
alsk = volpick.data.AlaskaDataset()
# file_handler = logging.FileHandler(filename=alsk.save_dir / "download.log")
# file_handler.setFormatter(
#     logging.Formatter(
#         "%(asctime)s | %(name)s | %(levelname)s | %(message)s",
#         datefmt="%Y-%m-%d %H:%M:%S",
#     ),
# )
# logger.addHandler(stream_handler)
# logger.addHandler(file_handler)

t1 = time.perf_counter()
# alsk = volpick.data.AlaskaDataset()

catalog_table = pd.read_csv(alsk.save_dir / "mseed_log" / "downloads.csv")

lp_table = catalog_table[catalog_table["source_type"] == "lp"].copy()
rg_table = catalog_table[catalog_table["source_type"] != "lp"].copy()

# destination path
# dest_dir = "/home/zhongyiyuan/DATA/my_datasets_seisbench/alaska"
dest_dir = "./test_alaska"
logger.info(f"Start to convert lp waveforms ...")
# convert lp waveforms
# volpick.data.convert_mseed_to_seisbench(
#     lp_table,
#     mseed_dir=alsk.save_dir / "mseed",
#     dest_dir=dest_dir,
#     chunk="_ak_lp",
#     skip_spikes=True,
#     split_prob=[0.85, 0.05, 0.1],
# )

# Convert the same number of regular earthquake waveform traces
n_lp = len(lp_table)
n_rg = len(rg_table)
assert n_rg > n_lp

logger.info(f"Start to convert rg waveforms ...")

lp_ids = lp_table.drop_duplicates(
    subset="source_id", keep="first", ignore_index=True, inplace=False
)["source_id"]
rg_ids = rg_table.drop_duplicates(
    subset="source_id", keep="first", ignore_index=True, inplace=False
)["source_id"]
n_lp_events = len(lp_ids)
n_rg_events = len(rg_ids)
assert n_rg_events > n_lp_events

The default saving directory is used: /home/zhongyiyuan/DATA/my_data/Alaska
/home/zhongyiyuan/DATA/my_data/Alaska exists


In [3]:
# select the same number of events
rand_events_idxs = np.sort(
    np.random.default_rng(seed=50).choice(rg_ids, size=n_lp_events, replace=False)
)

In [4]:
rand_events_idxs

array(['alk60002503', 'alk60002903', 'alk60003003', ..., 'alk90265357',
       'alk90265472', 'alk90266342'], dtype=object)

In [14]:
catalog_table

,source_id,source_origin_time,source_latitude_deg,source_longitude_deg,source_depth_km,source_magnitude,source_magnitude_type,source_type,station_network_code,station_code,...,trace_s_max_weight,trace_p_first_motion,station_latitude_deg,station_longitude_deg,station_elevation_m,station_epicentral_distance_m,path_azimuth_deg,path_back_azimuth_deg,trace_name,trace_sampling_rate_hz
0,alk61728463,1997-09-15T03:29:52.400000Z,58.247500,-155.162167,3.32,0.70,ml,le,AV,ACH,...,1.10,NaN,58.210000,-155.32810,960.0,10606.491596,246.879543,66.738473,alk61728463_AV.ACH..EH.mseed,100
1,alk61728463,1997-09-15T03:29:52.400000Z,58.247500,-155.162167,3.32,0.70,ml,le,AV,KBM,...,0.83,D,58.274670,-155.20517,687.0,3940.899433,320.184846,140.148274,alk61728463_AV.KBM..EH.mseed,100
2,alk61728463,1997-09-15T03:29:52.400000Z,58.247500,-155.162167,3.32,0.70,ml,le,AV,KCG,...,1.10,D,58.306900,-155.11350,762.0,7206.045462,23.324389,203.365785,alk61728463_AV.KCG..EH.mseed,100
3,alk61728483,1997-09-15T16:56:35.910000Z,58.247000,-155.159000,2.77,0.70,ml,le,AV,ACH,...,1.06,NaN,58.210000,-155.32810,960.0,10756.369450,247.544011,67.400250,alk61728483_AV.ACH..EH.mseed,100
4,alk61728483,1997-09-15T16:56:35.910000Z,58.247000,-155.159000,2.77,0.70,ml,le,AV,KBM,...,1.06,U,58.274670,-155.20517,687.0,4104.147397,318.691033,138.651768,alk61728483_AV.KBM..EH.mseed,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
389004,alk70401514,2014-12-10T00:10:32.060000Z,54.120500,-165.959833,5.22,0.33,ml,le,AV,AHB,...,1.08,U,54.115339,-165.81610,411.0,9416.304432,93.439360,273.555817,alk70401514_AV.AHB..EH.mseed,100
389005,alk70429614,2015-02-04T22:53:16.930000Z,54.119667,-165.728000,12.70,0.41,ml,le,AV,ZRO,...,1.06,D,54.091630,-165.97798,441.7,16646.221330,259.296072,79.093563,alk70429614_AV.ZRO..EH.mseed,100
389006,alk61195031,2016-04-23T05:10:49.470000Z,61.067500,-152.074500,14.90,0.33,ml,le,AV,CKT,...,1.05,U,61.200200,-152.20850,975.0,16455.389510,334.034244,153.916893,alk61195031_AV.CKT..EH.mseed,100
389007,alk61504061,2017-06-23T12:10:19.670000Z,61.309833,-152.247500,0.84,-0.03,ml,le,AV,BGL,...,0.78,D,61.266300,-152.39130,1127.0,9109.257621,237.885520,57.759401,alk61504061_AV.BGL..EH.mseed,100


In [5]:
rand_events_rg_table = rg_table[rg_table["source_id"].isin(rand_events_idxs)].copy()

In [12]:
rg_table[["source_id", "source_origin_time"]]

,source_id,source_origin_time
0,alk61728463,1997-09-15T03:29:52.400000Z
1,alk61728463,1997-09-15T03:29:52.400000Z
2,alk61728463,1997-09-15T03:29:52.400000Z
3,alk61728483,1997-09-15T16:56:35.910000Z
4,alk61728483,1997-09-15T16:56:35.910000Z
...,...,...
389004,alk70401514,2014-12-10T00:10:32.060000Z
389005,alk70429614,2015-02-04T22:53:16.930000Z
389006,alk61195031,2016-04-23T05:10:49.470000Z
389007,alk61504061,2017-06-23T12:10:19.670000Z


In [13]:
rand_events_rg_table["source_origin_time"]

37        1997-09-21T23:33:08.870000Z
38        1997-09-21T23:54:57.950000Z
60        1997-09-23T06:52:53.200000Z
66        1997-09-23T10:23:34.400000Z
67        1997-09-23T10:23:34.400000Z
                     ...             
388974    1999-11-11T12:34:52.500000Z
388979    2005-10-07T19:12:55.290000Z
388996    2009-11-14T18:53:49.620000Z
389004    2014-12-10T00:10:32.060000Z
389006    2016-04-23T05:10:49.470000Z
Name: source_origin_time, Length: 64263, dtype: object

In [ ]:

# select the same number of waveforms
rand_waveform_idxs = np.sort(
    np.random.default_rng(seed=100).choice(
        len(rand_events_rg_table), size=n_lp, replace=False
    )
)

randomly_selected_rg_table = rand_events_rg_table.iloc[rand_waveform_idxs].copy()
print(rand_events_idxs)
np.savetxt("rand_events_idxs.txt", rand_events_idxs, fmt="%s")
np.savetxt("rand_waveform_idxs.txt", rand_waveform_idxs)
rg_table.to_csv("x0.csv", index=False)
rand_events_rg_table.to_csv("x1.csv", index=False)
randomly_selected_rg_table.to_csv("x.csv", index=False)
# volpick.data.convert_mseed_to_seisbench(
#     randomly_selected_rg_table,
#     mseed_dir=alsk.save_dir / "mseed",
#     dest_dir=dest_dir,
#     chunk="_ak_rg",
#     skip_spikes=True,
#     split_prob=[0.85, 0.05, 0.1],
# )
t2 = time.perf_counter()
running_time = str(datetime.timedelta(seconds=t2 - t1))
logger.info(f"Finish format conversion. Running time: {running_time}")